# SLM: Wiki40B日本語データセットでのDiffusionモデル学習

このノートブックでは、日本語Wiki40Bデータセットを使用してWave Network言語モデルをDiffusionアプローチで学習します。

## 1. 環境セットアップ

In [ ]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# プロジェクトのクローンとインストール
!git clone https://github.com/yourusername/slm_project.git
!cd slm_project && pip install -e .

In [ ]:
# 必要なライブラリのインストール
!pip install sentencepiece datasets transformers==4.31.0 accelerate==0.21.0 pywavelets==1.4.1 bitsandbytes

## 2. Wiki40B日本語データセットの前処理

In [ ]:
import os
from slm.wiki40b_ja_dataset import prepare_dataset, train_tokenizer

# Google Drive上のデータディレクトリ
data_dir = "/content/drive/MyDrive/slm/data/wiki40b_ja"
os.makedirs(data_dir, exist_ok=True)

# データセットをダウンロードして前処理
train_path, valid_path, test_path = prepare_dataset(data_dir)
print(f"データセット前処理完了。保存パス: {train_path}, {valid_path}, {test_path}")

In [ ]:
# SentencePieceトークナイザーの学習
model_prefix = "sp_jwiki"
vocab_size = 32000

train_tokenizer(train_path, valid_path, data_dir, model_prefix, vocab_size)
print(f"トークナイザーの学習完了。モデルファイル: {os.path.join(data_dir, model_prefix)}.model")

In [ ]:
# トークナイザーのテスト
from slm.wiki40b_ja_dataset import load_tokenizer, test_tokenizer_functionality

tokenizer = load_tokenizer(data_dir, model_prefix)
test_tokenizer_functionality(tokenizer, "これはトークナイザーのテストです。日本語Wikipediaで学習されたモデルを使います。")

# [MASK]トークンのIDを確認
mask_id = tokenizer.piece_to_id("[MASK]")
print(f"[MASK]トークンID: {mask_id}")

## 3. モデルの学習

In [ ]:
# Diffusionモデル学習の実行
!cd /content/slm_project && python slm/train_wiki40b_ja_diffusion.py \
    --data_dir="/content/drive/MyDrive/slm/data/wiki40b_ja" \
    --output_dir="/content/drive/MyDrive/slm/outputs" \
    --model_prefix="sp_jwiki" \
    --hidden_size=1024 \
    --num_layers=3 \
    --max_seq_len=512 \
    --batch_size=8 \
    --epochs=3 \
    --learning_rate=2e-5

## 4. 学習結果の確認

In [ ]:
import os
import torch
from slm.modules.wave_network import WaveNetworkLM
from slm.config import ModelConfig
from slm.wiki40b_ja_dataset import load_tokenizer

# モデルチェックポイントパス
output_dir = "/content/drive/MyDrive/slm/outputs"
run_dirs = [d for d in os.listdir(output_dir) if os.path.isdir(os.path.join(output_dir, d)) and d.startswith("wiki40b_ja_diffusion")]
run_dirs.sort()
latest_run = run_dirs[-1] if run_dirs else None

if latest_run:
    checkpoint_dir = os.path.join(output_dir, latest_run, "checkpoints")
    checkpoint_files = [f for f in os.listdir(checkpoint_dir) if f.endswith(".pt")]
    checkpoint_files.sort()
    latest_checkpoint = os.path.join(checkpoint_dir, checkpoint_files[-1]) if checkpoint_files else None
    
    print(f"最新の実行: {latest_run}")
    print(f"利用可能なチェックポイント: {checkpoint_files}")
    print(f"最新のチェックポイント: {latest_checkpoint}")
else:
    print("学習済みのモデルが見つかりません")

In [ ]:
# 学習済みモデルのロード（上記で最新のチェックポイントが見つかった場合）
if 'latest_checkpoint' in locals() and latest_checkpoint:
    # トークナイザーのロード
    data_dir = "/content/drive/MyDrive/slm/data/wiki40b_ja"
    model_prefix = "sp_jwiki"
    tokenizer = load_tokenizer(data_dir, model_prefix)
    
    # チェックポイントのロード
    checkpoint = torch.load(latest_checkpoint, map_location='cpu')
    model_config = checkpoint["model_config"]
    
    # トークナイザーをモデルに設定
    model_config.set_tokenizer(tokenizer)
    
    # モデルのインスタンス化と重みのロード
    model = WaveNetworkLM(model_config)
    model.load_state_dict(checkpoint["model_state_dict"])
    
    print(f"モデルを正常にロードしました。パラメータ数: {sum(p.numel() for p in model.parameters()):,}")
else:
    print("学習済みモデルがロードできませんでした")

## 5. 簡単な推論テスト

In [ ]:
# 簡単な推論テスト（モデルがロードされている場合）
if 'model' in locals() and 'tokenizer' in locals():
    from slm.diffusion import SimpleTextDiffusion
    import torch
    
    # デバイスの設定
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    
    # マスクトークンIDの取得
    mask_token_id = tokenizer.piece_to_id("[MASK]")
    
    # Diffusionモデルのインスタンス化
    diffuser = SimpleTextDiffusion(
        timesteps=20,
        mask_token_id=mask_token_id,
        vocab_size=tokenizer.get_piece_size()
    ).to(device)
    
    # テスト文の用意
    test_text = "日本は四季折々の自然が美しい国です。"
    print(f"元のテキスト: {test_text}")
    
    # トークン化
    tokens = tokenizer.encode(test_text, out_type=int)
    token_tensor = torch.tensor([tokens], device=device)
    
    # 完全にノイズを加える（最大タイムステップ)
    t = torch.tensor([diffuser.timesteps - 1], device=device)
    noisy_tokens, _ = diffuser(token_tensor, t)
    
    # ノイズを加えたテキストの表示
    noisy_text = tokenizer.decode(noisy_tokens[0].cpu().tolist())
    print(f"ノイズを加えたテキスト: {noisy_text}")
    
    # 逐次的にノイズを除去する簡易版デノイズプロセス
    with torch.no_grad():
        current_tokens = noisy_tokens.clone()
        
        for timestep in reversed(range(diffuser.timesteps)):
            # マスクされた位置を見つける
            mask_positions = (current_tokens == mask_token_id)
            
            if not mask_positions.any():
                break
                
            # モデルの予測を取得
            logits = model(current_tokens)
            
            # マスクされた位置でのトップk予測を取得
            k = 5
            topk_probs, topk_indices = torch.topk(logits.softmax(dim=-1), k, dim=-1)
            
            # マスクごとにトップ1の予測で置き換え
            for i in range(current_tokens.size(0)):
                for j in range(current_tokens.size(1)):
                    if mask_positions[i, j]:
                        # ここではトップ1の予測を使用
                        current_tokens[i, j] = topk_indices[i, j, 0]
                        
            # 結果を表示
            if timestep % 5 == 0 or timestep == 0:
                current_text = tokenizer.decode(current_tokens[0].cpu().tolist())
                print(f"タイムステップ {timestep} での復元: {current_text}")
        
        final_text = tokenizer.decode(current_tokens[0].cpu().tolist())
        print(f"\n最終的な復元テキスト: {final_text}")
else:
    print("モデルがロードされていないため、推論テストを実行できません")